# Function Calling
- Function Calling은 OpenAI API에서 특정 함수 정의(JSON schema 기반)를 주고, 모델이 해당 함수를 호출하는 형식의 응답을 생성하게 하는 기능입니다.
- 챗봇, 에이전트(Agent), 도구(Tool) 사용 등 다양한 자동화 및 플러그인 연동에 활용됩니다.

- 지원 모델 gpt-3-turbo, gpt-4, gpt-4-turbo  (2025.1 현재)

- 모델이 우리의 코드를 호출하도록 해서 우리의 함수들을 모델이 호출할수 있도록 하거나
- 모델이 우리가 원하는 특정 모양과 형식의 output 을 갖도록 강제할수 있다.

- https://platform.openai.com/docs/guides/function-calling?api-mode=chat


In [1]:
# 모델로 하여금 우리가 원하는 구체적인 형태의 output 을 강제하는 방법들

# 1. example
# 2. function calling

In [2]:
import os 

from dotenv import load_dotenv
load_dotenv()

print(f'OPENAI_API_KEY={os.getenv("OPENAI_API_KEY")[:20]}...')

OPENAI_API_KEY=sk-proj-iKU13YeoxNgF...


In [3]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate

In [4]:
llm = ChatOpenAI(
    temperature=0.1
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")

chain = prompt | llm

response = chain.invoke({'city': 'rome'})

print(response.content)

I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Rome.


In [5]:
#  ↑ 모델은 실시간 정보에 대한 답변이 불가능하다!

In [18]:
# 만약! 우리가 실시간 날씨 정보를 가져오는 함수가 있다면!

# 특정 위도,경도 좌표가 주어지면 현재 날씨를 가져오는 함수 (가정)
def get_weather(lon, lat):
    print(f'💛call an weahter api... lon:{lon}, lat:{lat}')  



In [7]:
# 우리가 함수 호출로 할 수 있는 일은 앞선 실망스러운(?) 응답을 받는 대신에 
# gpt-3, gpt-4 에게 우리에게 '이러이런한 일'을 해주는 get_weather() 라는 함수가 있다고 말해주는 거다.
# 그러면, gpt 는 우리의 질문을 보게 되겠고, 함수를 사용하는 것이 이런 실망스러운 답변을 주는 것 보다 나은지 판별할거다.

# 어떻게 GPT 나 LLM 을 그렇게 동작하도록 만들수 있을까?
# LLM 이 get_weather 라는 함수가 있다는 걸 알도록 할수 있을까?

# 이 함수를 위한 JSON scheme 를 만드는 거다!

### JSON schema 란?

- https://json-schema.org/overview/what-is-jsonschema


# 함수의 스키마

In [8]:
function_schema = {
    "name": "get_weather", # 함수의 이름

    # 무슨일을 하는 함수인지 기술
    "description": "function that takes longitude and latitude to find the weather of a place",  
    
    "parameters": { # 함수의 파라미터 기술
        "type": "object",
        "properties": {
            "lon": {"type": "string", "description": "The longitude coordinate"},
            "lat": {"type": "string", "description": "The latitude coordinate"},
        },
    },
    
    "required": ["lon", "lat"] # 필수 파라미터 기술
}

# ChatOpenAI 에 함수 전달하기

In [21]:
llm = ChatOpenAI(
    temperature=0.1
).bind(   # <- 여기에 ChatOpenAI 에게 전달할 인수를 추가해줄수 있다.
    functions=[    # <- 원하는 만큼 많은 함수를 전달할수 있다.
        function_schema,  # 준비한 함수 schema 전달
    ],

    # ↓기본적으로 모델이 특정 함수를 사용하도록 '강제'하거나, 
    #   모델이 함수를 사용하도록 하거나 그냥 답변을 할 수 있도록 모델 스스로 선택하게 할수 있다.

    # 모델을 강제로 함수를 사용하도록 하려면! 아래와 같이 해야 한다
    # function_call={
    #     "name": "get_weather",   # <-- get_weather 함수를 반드시 사용하라!
    # },    

    # 만약 AI 에게 선택권을 주어서 AI 가 필요에 따라 사용하게 하려면?
    function_call="auto"
)

In [22]:
chain = prompt | llm

response = chain.invoke({'city': 'rome'})

# print(response.content)  # 내용 없다.

response  # AIMessage

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 74, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Co7XDrhCMfRwDVFS6OLWx1BpLfU07', 'finish_reason': 'function_call', 'logprobs': None}, id='run--5686d193-15db-4490-8fb7-a2733f0444d3-0', usage_metadata={'input_tokens': 74, 'output_tokens': 24, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
"""
AIMessage(
    content='',   # <-- 응답은 받았지만 content 는 없다.  대신! 함수가 호출된것을 볼수 있다!
    additional_kwargs={
        # ↓ 함수가 호출되었다!  
        # 모델은 get_weather 라는 함수를 호출해 줬으면 해!  라고 말하고 있는거다.
        # 그리고, "네가 경도와 위도에 대한 argument 를 제공해 줬으면 해" 라고 하고 있는 거다.        
        # rome 의 lon 과 lat 값을 전달해 주었다!  오오오오!
        'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}, 
        'refusal': None
    }, 
"""
None

In [12]:
# 함수의 인자 값들
response.additional_kwargs

{'function_call': {'arguments': '{"lon":"12.4964","lat":"41.9028"}',
  'name': 'get_weather'},
 'refusal': None}

In [13]:
response.additional_kwargs['function_call']

{'arguments': '{"lon":"12.4964","lat":"41.9028"}', 'name': 'get_weather'}

In [14]:
response.additional_kwargs['function_call']['arguments']

'{"lon":"12.4964","lat":"41.9028"}'

In [15]:
# 위 json 을 파이썬 객체로 변환
import json

r = json.loads(response.additional_kwargs['function_call']['arguments'])
r

{'lon': '12.4964', 'lat': '41.9028'}

In [16]:
r['lon'], r['lat']

('12.4964', '41.9028')

In [19]:
get_weather(r['lon'], r['lat'])

💛call an weahter api... lon:12.4964, lat:41.9028


In [20]:
get_weather(**r)

💛call an weahter api... lon:12.4964, lat:41.9028


# QuizGPT 의 퀴즈 생성하기 스키마

In [23]:
function_schema = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",  

    "parameters": {
        # 우리가 받기 원하는 포맷의 스키마
# 바로 이 형태를 정의하는 거다. 
# {
#   "questions":[
#      0:{
#        "question":"What … in the story?"
#        "answers":[
#           0:{
#             "answer":"John"
#             "correct":false
#           }
#           ... 
#        ]
#      }
#      ...
# }            
        "questions": {
            "type": "array",
            "items": {
               "type": "object",
               "properties": {
                   "question": {
                       "type": "string",
                   },
                   "answers": {
                       "type": "array",
                       "items": {
                           "type": "object",
                           "properties": {
                               "answer": {
                                   "type": "string",
                               },
                               "correct": {
                                   "type": "boolean",
                               },
                           },
                           "required": ["answer", "correct"], 
                       },
                   },
               },
                "required": ["question", "answers"],
            },
        },            
    },
    "required": ["questions"],
}

In [25]:
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
import json

function_schema = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",  
    
    "parameters": { 
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",   
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {  
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
    },
     "required": ["questions"],
}  


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    functions=[
        function_schema  # 여기에는 원하는 만큼의 function 들을 적어 넣을수 있다.
    ],
    # function_call="auto"  # "auto" 모델이 우리 함수를 사용할수도 있고 사용하지 않을수도 있다.
    function_call={
        "name": "create_quiz",
    },
)

# prompt 변경
prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

response = response.additional_kwargs['function_call']['arguments']

response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"1000 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built by the ancient Romans?","answers":[{"answer":"Eiffel Tower","correct":false},{"answer":"Colosseum","correct":true},{"answer":"Big Ben","correct":false}]}]}'